# OpenAI - Code Suggestions Quality and Developer Productivity

```SQL
CREATE TABLE fct_code_suggestions (
    suggestion_id INTEGER,
    developer_id INTEGER,
    programming_language VARCHAR,
    complexity_level VARCHAR,
    coding_speed_improvement DECIMAL,
    error_reduction_percentage DECIMAL, suggestion_date DATE
);

INSERT INTO fct_code_suggestions (
    suggestion_id,
    developer_id,
    programming_language,
    complexity_level,
    coding_speed_improvement,
    error_reduction_percentage,
    suggestion_date
)
VALUES
    (1, 101, 'Python', 'Easy', 12.5, 18, '2024-04-03'),
    (2, 102, 'Python', 'Medium', 10, 17.5, '2024-04-05'),
    (3, 103, 'Python', 'Hard', 9, 15, '2024-04-07'),
    (4, 104, 'JavaScript', 'Easy', 14, 13, '2024-04-02'),
    (5, 101, 'JavaScript', 'Medium', 16.5, 14.5, '2024-04-08'),
    (6, 105, 'JavaScript', 'Hard', 15, 12, '2024-04-12'),
    (7, 106, 'Java', 'Easy', 11, 16, '2024-04-03'),
    (8, 107, 'Java', 'Medium', 13, 20, '2024-04-06'),
    (9, 108, 'Java', 'Hard', 12, 19.5, '2024-04-10'),
    (10, 109, 'C++', 'Easy', 8, 14, '2024-04-04'),
    (11, 110, 'C++', 'Medium', 7.5, 11, '2024-04-09'),
    (12, 105, 'C++', 'Hard', 9.5, 16, '2024-04-11'),
    (13, 111, 'Python', 'Medium', 15, 19, '2024-04-15'),
    (14, 112, 'JavaScript', 'Easy', 18, 16, '2024-04-16'),
    (15, 113, 'Java', 'Medium', 17, 20, '2024-04-18');

SELECT * FROM fct_code_suggestions;
```

In [1]:
import pandas as pd
import numpy as np

In [3]:
df_code = pd.read_csv('Data/028/fct_code_suggestions.csv',parse_dates=['suggestion_date'])

df_code.head()

,suggestion_id,developer_id,programming_language,complexity_level,coding_speed_improvement,error_reduction_percentage,suggestion_date
0,1,101,Python,Easy,12.5,18.0,2024-04-03
1,2,102,Python,Medium,10.0,17.5,2024-04-05
2,3,103,Python,Hard,9.0,15.0,2024-04-07
3,4,104,JavaScript,Easy,14.0,13.0,2024-04-02
4,5,101,JavaScript,Medium,16.5,14.5,2024-04-08


# Pregunta 1

### ¿Cuál es el porcentaje promedio de mejora en la velocidad de codificación para cada lenguaje de programación en abril de 2024? Este análisis nos ayudará a determinar si las sugerencias actuales están impulsando efectivamente la velocidad de desarrollo.

In [5]:
df_abril = df_code[
    (df_code['suggestion_date'].between('2024-04-01','2024-04-30'))
]

respuesta1 = df_abril.groupby('programming_language')['coding_speed_improvement'].mean().reset_index()

respuesta1 = respuesta1.sort_values(by='coding_speed_improvement' ,ascending=False)

respuesta1

,programming_language,coding_speed_improvement
2,JavaScript,15.875000
1,Java,13.250000
3,Python,11.625000
0,C++,8.333333


```SQL
SELECT
    programming_language,
    AVG(coding_speed_improvement) AS avg_speed_improvement
FROM fct_code_suggestions
WHERE suggestion_date BETWEEN '2024-04-01' AND '2024-04-30'
GROUP BY programming_language
ORDER BY avg_speed_improvement DESC;
```

# Pregunta 2

### Para cada lenguaje de programación en abril de 2024, ¿cuál es el porcentaje mínimo de reducción de errores observado en todas las sugerencias de código generadas por IA? Esto ayudará a identificar los lenguajes donde la reducción de errores es insuficiente y podrían requerir mejoras específicas.

In [7]:
df_abril = df_code[
    (df_code['suggestion_date'].between('2024-04-01','2024-04-30'))
].copy()

respuesta2 = df_abril.groupby('programming_language')['error_reduction_percentage'].min().reset_index()

respuesta2 = respuesta2.sort_values(by='error_reduction_percentage', ascending=True)

respuesta2

,programming_language,error_reduction_percentage
0,C++,11.0
2,JavaScript,12.0
3,Python,15.0
1,Java,16.0


```SQL
SELECT
    programming_language,
    MIN(error_reduction_percentage) AS min_error_reduction
FROM fct_code_suggestions
WHERE suggestion_date BETWEEN '2024-04-01' AND '2024-04-30'
GROUP BY programming_language
ORDER BY min_error_reduction ASC;
```

# Pregunta 3

### Para abril de 2024, primero concatene el lenguaje de programación y el nivel de complejidad para formar un identificador único. Luego, utilizando el promedio de la mejora en la velocidad de programación y el porcentaje de reducción de errores como una métrica combinada, ¿qué combinación concatenada muestra la mayor mejora agregada? Este análisis final informa directamente los esfuerzos para lograr un aumento específico en la productividad de los desarrolladores y la reducción de errores.

In [8]:
# 1. Filtramos abril
df_april = df_code[df_code['suggestion_date'].between('2024-04-01', '2024-04-30')].copy()

# 2. Creamos la columna concatenada
df_april['lang_complexity'] = df_april['programming_language'] + '_' + df_april['complexity_level']

# 3. Agrupamos y calculamos la media de ambas columnas, luego su promedio
res = df_april.groupby('lang_complexity')[['coding_speed_improvement', 'error_reduction_percentage']].mean()
res['combined_score'] = res.mean(axis=1)

# 4. Obtenemos el ganador
ganador = res['combined_score'].sort_values(ascending=False).head(1)
print(ganador)

lang_complexity
Java_Medium    17.5
Name: combined_score, dtype: float64


```SQL
SELECT
    (programming_language || '_' || complexity_level),
    (AVG(coding_speed_improvement) + AVG(error_reduction_percentage))/2.0 AS avg_improvement
FROM fct_code_suggestions
WHERE suggestion_date BETWEEN '2024-04-01' AND '2024-04-30'
GROUP BY programming_language, complexity_level
ORDER BY  avg_improvement DESC
LIMIT 1;
```